# Flood Risk Analysis

In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap

flood_df = pd.read_csv("../../data/raw/AEGISDataset.csv")
# Check the data
flood_df.head()

,lat,lon,flood_heig,elevation,precipitat
0,14.640394,121.055708,0,54.553295,9.0
1,14.698299,121.002132,0,21.856272,10.0
2,14.698858,121.100261,0,69.322807,16.0
3,14.571310,120.983334,0,10.987241,8.0
4,14.762232,121.075735,0,87.889847,18.0


In [2]:
numeric_columns = ["lat","lon","flood_heig","elevation","precipitat"]

# Convert columns to numeric, coercing errors to NaN
flood_df[numeric_columns] = flood_df[numeric_columns].apply(pd.to_numeric, errors = 'coerce')

In [3]:
cleaned_flood_df = flood_df.dropna()

In [4]:
# Check the cleaned data
print(f"NaN after cleaning: {cleaned_flood_df.isna().sum()}")

NaN after cleaning: lat           0
lon           0
flood_heig    0
elevation     0
precipitat    0
dtype: int64


In [5]:
# prepare data for heatmap
prep_flood_df = cleaned_flood_df.copy()

In [6]:
# Normalize the flood height, elevation, and precipitation for better visualization
prep_flood_df["flood_heig_norm"] = (prep_flood_df["flood_heig"] - prep_flood_df["flood_heig"].min()) / (prep_flood_df["flood_heig"].max() - prep_flood_df["flood_heig"].min())
prep_flood_df["elevation_norm"] = (prep_flood_df["elevation"] - prep_flood_df["elevation"].min()) / (prep_flood_df["elevation"].max() - prep_flood_df["elevation"].min())
prep_flood_df["precipitat_norm"] = (prep_flood_df["precipitat"] - prep_flood_df["precipitat"].min()) / (prep_flood_df["precipitat"].max() - prep_flood_df["precipitat"].min())

In [7]:
prep_flood_df["elevation_norm"] = 1 - prep_flood_df["elevation_norm"]

prep_flood_df["risk_score"] = (prep_flood_df["flood_heig_norm"] * 0.4 + prep_flood_df["elevation_norm"] * 0.3 +  prep_flood_df["precipitat_norm"] * 0.3)


In [8]:
print("\nRisk Score for Each Location:")
for index, row in prep_flood_df.iterrows():
  print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}")


Risk Score for Each Location:
Latitude: 14.64039374, Longitude: 121.0557085, Flood Height: 0.0, Risk Score: 0.2503582609426578
Latitude: 14.69829935, Longitude: 121.0021322, Flood Height: 0.0, Risk Score: 0.3714576900771931
Latitude: 14.69885841, Longitude: 121.1002614, Flood Height: 0.0, Risk Score: 0.30210963466319884
Latitude: 14.57131006, Longitude: 120.9833345, Flood Height: 0.0, Risk Score: 0.3783929056101954
Latitude: 14.7622323, Longitude: 121.0757349, Flood Height: 0.0, Risk Score: 0.2700267817151137
Latitude: 14.6011784, Longitude: 121.0885987, Flood Height: 0.0, Risk Score: 0.4098931394636641
Latitude: 14.70803089, Longitude: 121.052674, Flood Height: 0.0, Risk Score: 0.3244220322521432
Latitude: 14.70920612, Longitude: 121.0502922, Flood Height: 0.0, Risk Score: 0.3445020220605377
Latitude: 14.70962531, Longitude: 121.0470059, Flood Height: 0.0, Risk Score: 0.3303591941495462
Latitude: 14.66933825, Longitude: 121.1328981, Flood Height: 0.0, Risk Score: 0.38514977754177726


In [9]:
def get_color(risk_score):
  if risk_score < 0.2:
    return 'green'
  elif 0.2 <= risk_score < 0.4:
    return 'yellow'
  elif 0.4 <= risk_score < 0.6:
    return 'orange'
  elif 0.6 <= risk_score < 0.8:
    return 'red'
  else:
    return 'darkred'

In [10]:
prep_flood_df['risk_color'] = prep_flood_df['risk_score'].apply(get_color)


In [11]:
manila_coordinates = [14.5995,120.9842]

In [12]:
map_flood_risk = folium.Map(location=manila_coordinates, zoom_start=12)

heat_data = prep_flood_df[['lat','lon','risk_score']].values.tolist()

HeatMap(heat_data, radius=15, max_zoom=13).add_to(map_flood_risk)

print("\nRisk Score for Each Location:")
for index, row in prep_flood_df.iterrows():
  print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}, Risk Color: {row['risk_color']}")


Risk Score for Each Location:
Latitude: 14.64039374, Longitude: 121.0557085, Flood Height: 0, Risk Score: 0.2503582609426578, Risk Color: yellow
Latitude: 14.69829935, Longitude: 121.0021322, Flood Height: 0, Risk Score: 0.3714576900771931, Risk Color: yellow
Latitude: 14.69885841, Longitude: 121.1002614, Flood Height: 0, Risk Score: 0.30210963466319884, Risk Color: yellow
Latitude: 14.57131006, Longitude: 120.9833345, Flood Height: 0, Risk Score: 0.3783929056101954, Risk Color: yellow
Latitude: 14.7622323, Longitude: 121.0757349, Flood Height: 0, Risk Score: 0.2700267817151137, Risk Color: yellow
Latitude: 14.6011784, Longitude: 121.0885987, Flood Height: 0, Risk Score: 0.4098931394636641, Risk Color: orange
Latitude: 14.70803089, Longitude: 121.052674, Flood Height: 0, Risk Score: 0.3244220322521432, Risk Color: yellow
Latitude: 14.70920612, Longitude: 121.0502922, Flood Height: 0, Risk Score: 0.3445020220605377, Risk Color: yellow
Latitude: 14.70962531, Longitude: 121.0470059, Floo

In [13]:
from IPython.display import display
display(map_flood_risk)